# Introduction

To use this module, install the following libraries:
<li> pandas 2.2.0 or higher </li>
<li> sqlalchemy 2.0 or higher </li>
<li> pathlib </li>
<li> pyexcelerate </li>
<li> openpyxl </li>

## Import module and set initial conditions

In [1]:
import py_sql
from py_sql import *
import warnings

# Display all columns in dataframe
pd.options.display.max_columns = None
# Ignore all warnings
warnings.filterwarnings('ignore')

schema = 'HCPH_RENNIELBAUTISTA'
my_credentials = get_credentials('my_credentials.txt',schema)

C:\Users\Renniel.Bautista\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Folder path C:\Users\Renniel.Bautista\Documents\Python-Oracle\Excel does not exists. Creating new directory.


# Generate SQL Script

This section showcases three functions that outputs SQL statements as string for: creating TABLE object, merging to existing table with pre-defined query, and creating PACKAGE object.

## Create TABLE

### Pre-requisites
Define table_name, sample query with rownum filter, and generate dataframe.

In [2]:
table_name = 'AH_RBB_POS_EC_SCORECARD_MONITORING'

In [3]:
query = f"""
with tab_base as
(
select
       dm.id_cuid
      -- Basic Scoring
      ,case
            when dm.text_contract_number is not null
            then 1 else 0
       end                                                                      flag_with_app
      ,dm.text_contract_number
      ,dm.date_application
      ,dm.pure_population
      ,dm.pure_population_pre
      ,dm.prelim1_strategy_type
      ,dm.risk_grade_prelim1
      ,dm.flag_with_offer_v2_prelim1
      ,dm.flag_with_offer_v2
      -- DPOS Tags
      ,dm.flag_with_dpos_offer_prelim1
      ,dm.flag_with_dpos_offer
      ,dm.code_prm_decision
      ,case when bs.text_contract_number is not null then 1
            when dm.text_contract_number is not null then 0
       end                                                                      flag_dpos_app
      -- Basic Scoring Decision
      ,dm.cnt_approve                                                           flag_approve
      ,dm.cnt_decided                                                           flag_decided
      ,dm.cnt_signed                                                            flag_signed
      -- Basic Scoring Scores
      ,sa.*
from ap_risk.pos_ec_datamart_tree dm
left join ap_risk.dpos_basic_scoring bs
       on bs.text_contract_number = dm.text_contract_number
      and bs.date_application = date '2024-12-20'
left join ap_risk.uw_bi_scorecard_dtm sa
       on sa.skp_credit_case = dm.skp_credit_case
      and sa.date_decision = date '2024-12-20'
      and sa.scorecard_name in (
                               'EC_202201_GMv8_Main_Scorecard'
                              ,'EC_202110_GMv8_Prelim_Scorecard'
                              ,'EC_202301_GMv10_Prelim_Scorecard'
                              ,'EC_202301_GMv10_Main_Scorecard'
                              )
where 1=1
      and dm.date_application = date '2024-12-20'
      and rownum <= 5
)

, tab_gm_pillar as
(
select
       tb.*
from tab_base
unpivot(
        (pillar_name,num_score_value_linear_pillar,num_score_value_pillar)
        for num_pillar_code
        in (
            (pillar_1,   pillar_value_linear_1,  pillar_value_1)    as '1'
           ,(pillar_2,   pillar_value_linear_2,  pillar_value_2)    as '2'
           ,(pillar_3,   pillar_value_linear_3,  pillar_value_3)    as '3'
           ,(pillar_4,   pillar_value_linear_4,  pillar_value_4)    as '4'
           ,(pillar_5,   pillar_value_linear_5,  pillar_value_5)    as '5'
           ,(pillar_6,   pillar_value_linear_6,  pillar_value_6)    as '6'
           ,(pillar_7,   pillar_value_linear_7,  pillar_value_7)    as '7'
           ,(pillar_8,   pillar_value_linear_8,  pillar_value_8)    as '8'
           ,(pillar_9,   pillar_value_linear_9,  pillar_value_9)    as '9'
           )
       ) tb
)

select
       count(1) over(partition by scom.id_cuid, scom.skp_credit_case) cnt_predictor_in_skp
      ,count(1) over(partition by scom.id_cuid, scom.skp_credit_case, scom.scorecard_name) cnt_predictor_in_sco
      ,count(1) over(partition by scom.id_cuid, scom.skp_credit_case, scom.scorecard_name, scom.pillar_name) cnt_predictor_in_pil
      ,scom.*
      ,scop.num_predictor_code
      ,scop.predictor_name
      ,scop.predictor_value                                                     num_score_value_linear_predictor
      ,trunc(1/(1+exp(-(scop.predictor_value))),6)                              num_score_value_predictor
      ,scop.predictor_bin
from tab_gm_pillar scom
left join (
            select * from ap_risk.uw_bi_scorecard_predictors_dtm
            unpivot(
                    (predictor_name,predictor_value,predictor_bin)
                    for num_predictor_code
                    in (
                        (predictor_1,   predictor_score_1,  predictor_bin_1)    as '1'
                       ,(predictor_2,   predictor_score_2,  predictor_bin_2)    as '2'
                       ,(predictor_3,   predictor_score_3,  predictor_bin_3)    as '3'
                       ,(predictor_4,   predictor_score_4,  predictor_bin_4)    as '4'
                       ,(predictor_5,   predictor_score_5,  predictor_bin_5)    as '5'
                       ,(predictor_6,   predictor_score_6,  predictor_bin_6)    as '6'
                       ,(predictor_7,   predictor_score_7,  predictor_bin_7)    as '7'
                       ,(predictor_8,   predictor_score_8,  predictor_bin_8)    as '8'
                       ,(predictor_9,   predictor_score_9,  predictor_bin_9)    as '9'
                       ,(predictor_10,  predictor_score_10, predictor_bin_10)   as '10'
                       ,(predictor_11,  predictor_score_11, predictor_bin_11)   as '11'
                       ,(predictor_12,  predictor_score_12, predictor_bin_12)   as '12'
                       ,(predictor_13,  predictor_score_13, predictor_bin_13)   as '13'
                       ,(predictor_14,  predictor_score_14, predictor_bin_14)   as '14'
                       ,(predictor_15,  predictor_score_15, predictor_bin_15)   as '15'
                       ,(predictor_16,  predictor_score_16, predictor_bin_16)   as '16'
                       ,(predictor_17,  predictor_score_17, predictor_bin_17)   as '17'
                       ,(predictor_18,  predictor_score_18, predictor_bin_18)   as '18'
                       )
                   )
            where 1=1
                  and date_decision = date '2024-12-20'
          ) scop
       on scop.skp_credit_case = scom.skp_credit_case
      and scop.date_decision = scom.date_decision
      and scop.skp_strategy_phase = scom.skp_strategy_phase
      and scop.pillar_name = scom.pillar_name
"""

In [4]:
# Apply Oracle hint in the query if none. This step is optional.
query = apply_ora_hint(query)

In [5]:
# Open
engine = connect_sqlalchemy(my_credentials)
conn = engine.raw_connection()
curr = conn.cursor()

df = pd.read_sql_query(query,engine)
df.columns = df.columns.str.upper()

# CLose
curr.close
engine.dispose()

### Generate CREATE TABLE script with partition by range

In [6]:
create_table_query = query_create_table(df,table_name,schema,engine)
print(create_table_query)

Do you want to create a partition by range? [YES/NO]  |  y
Input column for date_reference.  |  date_application


CREATE TABLE HCPH_RENNIELBAUTISTA.AH_RBB_POS_EC_SCORECARD_MONITORING
(
CNT_PREDICTOR_IN_SKP NUMBER, 
CNT_PREDICTOR_IN_SCO NUMBER, 
CNT_PREDICTOR_IN_PIL NUMBER, 
ID_CUID NUMBER, 
FLAG_WITH_APP NUMBER, 
TEXT_CONTRACT_NUMBER VARCHAR2(80 CHAR), 
DATE_APPLICATION DATE, 
PURE_POPULATION NUMBER, 
PURE_POPULATION_PRE NUMBER, 
PRELIM1_STRATEGY_TYPE VARCHAR2(80 CHAR), 
RISK_GRADE_PRELIM1 VARCHAR2(80 CHAR), 
FLAG_WITH_OFFER_V2_PRELIM1 NUMBER, 
FLAG_WITH_OFFER_V2 NUMBER, 
FLAG_WITH_DPOS_OFFER_PRELIM1 NUMBER, 
FLAG_WITH_DPOS_OFFER NUMBER, 
CODE_PRM_DECISION VARCHAR2(80 CHAR), 
FLAG_DPOS_APP NUMBER, 
FLAG_APPROVE NUMBER, 
FLAG_DECIDED NUMBER, 
FLAG_SIGNED NUMBER, 
SKP_CREDIT_CASE NUMBER, 
DATE_DECISION DATE, 
FLAG_DECIDING_SCORECARD NUMBER, 
DECIDING_SCORECARD_VALUE VARCHAR2(80 CHAR), 
SCORECARD_NAME VARCHAR2(80 CHAR), 
NUM_SCORE_VALUE_LINEAR NUMBER, 
NUM_SCORE_VALUE NUMBER, 
SKP_STRATE

### Generate CREATE TABLE script without partition by range

In [7]:
create_table_query = query_create_table(df,table_name,schema,engine)
print(create_table_query)

Do you want to create a partition by range? [YES/NO]  |  n

CREATE TABLE HCPH_RENNIELBAUTISTA.AH_RBB_POS_EC_SCORECARD_MONITORING
(
CNT_PREDICTOR_IN_SKP NUMBER, 
CNT_PREDICTOR_IN_SCO NUMBER, 
CNT_PREDICTOR_IN_PIL NUMBER, 
ID_CUID NUMBER, 
FLAG_WITH_APP NUMBER, 
TEXT_CONTRACT_NUMBER VARCHAR2(80 CHAR), 
DATE_APPLICATION DATE, 
PURE_POPULATION NUMBER, 
PURE_POPULATION_PRE NUMBER, 
PRELIM1_STRATEGY_TYPE VARCHAR2(80 CHAR), 
RISK_GRADE_PRELIM1 VARCHAR2(80 CHAR), 
FLAG_WITH_OFFER_V2_PRELIM1 NUMBER, 
FLAG_WITH_OFFER_V2 NUMBER, 
FLAG_WITH_DPOS_OFFER_PRELIM1 NUMBER, 
FLAG_WITH_DPOS_OFFER NUMBER, 
CODE_PRM_DECISION VARCHAR2(80 CHAR), 
FLAG_DPOS_APP NUMBER, 
FLAG_APPROVE NUMBER, 
FLAG_DECIDED NUMBER, 
FLAG_SIGNED NUMBER, 
SKP_CREDIT_CASE NUMBER, 
DATE_DECISION DATE, 
FLAG_DECIDING_SCORECARD NUMBER, 
DECIDING_SCORECARD_VALUE VARCHAR2(80 CHAR), 
SCORECARD_NAME VARCHAR2(80 CHAR), 
NUM_SCORE_VALUE_LINEAR NUMBER, 
NUM_SCORE_VALUE NUMBER, 
SKP_STRATEGY_PHASE NUMBER, 
NUM_PILLAR_CODE VARCHAR2(80 CHAR), 
P

## Merge TABLE

### Pre-requisites
Define query with date parameters and connection key/s for merging

In [8]:
query_c_date = f"""
with tab_base as
(
select
       dm.id_cuid
      -- Basic Scoring
      ,case
            when dm.text_contract_number is not null
            then 1 else 0
       end                                                                      flag_with_app
      ,dm.text_contract_number
      ,dm.date_application
      ,dm.pure_population
      ,dm.pure_population_pre
      ,dm.prelim1_strategy_type
      ,dm.risk_grade_prelim1
      ,dm.flag_with_offer_v2_prelim1
      ,dm.flag_with_offer_v2
      -- DPOS Tags
      ,dm.flag_with_dpos_offer_prelim1
      ,dm.flag_with_dpos_offer
      ,dm.code_prm_decision
      ,case when bs.text_contract_number is not null then 1
            when dm.text_contract_number is not null then 0
       end                                                                      flag_dpos_app
      -- Basic Scoring Decision
      ,dm.cnt_approve                                                           flag_approve
      ,dm.cnt_decided                                                           flag_decided
      ,dm.cnt_signed                                                            flag_signed
      -- Basic Scoring Scores
      ,sa.*
from ap_risk.pos_ec_datamart_tree dm
left join ap_risk.dpos_basic_scoring bs
       on bs.text_contract_number = dm.text_contract_number
      and bs.date_application >= c_date_start
      and bs.date_application <  c_date_end
left join ap_risk.uw_bi_scorecard_dtm sa
       on sa.skp_credit_case = dm.skp_credit_case
      and sa.date_decision >= c_date_start
      and sa.date_decision <  c_date_end + 14
      and sa.scorecard_name in (
                               'EC_202201_GMv8_Main_Scorecard'
                              ,'EC_202110_GMv8_Prelim_Scorecard'
                              ,'EC_202301_GMv10_Prelim_Scorecard'
                              ,'EC_202301_GMv10_Main_Scorecard'
                              )
where 1=1
      and dm.date_application >= c_date_start
      and dm.date_application <  c_date_end
)

, tab_gm_pillar as
(
select
       tb.*
from tab_base
unpivot(
        (pillar_name,num_score_value_linear_pillar,num_score_value_pillar)
        for num_pillar_code
        in (
            (pillar_1,   pillar_value_linear_1,  pillar_value_1)    as '1'
           ,(pillar_2,   pillar_value_linear_2,  pillar_value_2)    as '2'
           ,(pillar_3,   pillar_value_linear_3,  pillar_value_3)    as '3'
           ,(pillar_4,   pillar_value_linear_4,  pillar_value_4)    as '4'
           ,(pillar_5,   pillar_value_linear_5,  pillar_value_5)    as '5'
           ,(pillar_6,   pillar_value_linear_6,  pillar_value_6)    as '6'
           ,(pillar_7,   pillar_value_linear_7,  pillar_value_7)    as '7'
           ,(pillar_8,   pillar_value_linear_8,  pillar_value_8)    as '8'
           ,(pillar_9,   pillar_value_linear_9,  pillar_value_9)    as '9'
           )
       ) tb
)

select
       count(1) over(partition by scom.id_cuid, scom.skp_credit_case) cnt_predictor_in_skp
      ,count(1) over(partition by scom.id_cuid, scom.skp_credit_case, scom.scorecard_name) cnt_predictor_in_sco
      ,count(1) over(partition by scom.id_cuid, scom.skp_credit_case, scom.scorecard_name, scom.pillar_name) cnt_predictor_in_pil
      ,scom.*
      ,scop.num_predictor_code
      ,scop.predictor_name
      ,scop.predictor_value                                                     num_score_value_linear_predictor
      ,trunc(1/(1+exp(-(scop.predictor_value))),6)                              num_score_value_predictor
      ,scop.predictor_bin
from tab_gm_pillar scom
left join (
            select * from ap_risk.uw_bi_scorecard_predictors_dtm
            unpivot(
                    (predictor_name,predictor_value,predictor_bin)
                    for num_predictor_code
                    in (
                        (predictor_1,   predictor_score_1,  predictor_bin_1)    as '1'
                       ,(predictor_2,   predictor_score_2,  predictor_bin_2)    as '2'
                       ,(predictor_3,   predictor_score_3,  predictor_bin_3)    as '3'
                       ,(predictor_4,   predictor_score_4,  predictor_bin_4)    as '4'
                       ,(predictor_5,   predictor_score_5,  predictor_bin_5)    as '5'
                       ,(predictor_6,   predictor_score_6,  predictor_bin_6)    as '6'
                       ,(predictor_7,   predictor_score_7,  predictor_bin_7)    as '7'
                       ,(predictor_8,   predictor_score_8,  predictor_bin_8)    as '8'
                       ,(predictor_9,   predictor_score_9,  predictor_bin_9)    as '9'
                       ,(predictor_10,  predictor_score_10, predictor_bin_10)   as '10'
                       ,(predictor_11,  predictor_score_11, predictor_bin_11)   as '11'
                       ,(predictor_12,  predictor_score_12, predictor_bin_12)   as '12'
                       ,(predictor_13,  predictor_score_13, predictor_bin_13)   as '13'
                       ,(predictor_14,  predictor_score_14, predictor_bin_14)   as '14'
                       ,(predictor_15,  predictor_score_15, predictor_bin_15)   as '15'
                       ,(predictor_16,  predictor_score_16, predictor_bin_16)   as '16'
                       ,(predictor_17,  predictor_score_17, predictor_bin_17)   as '17'
                       ,(predictor_18,  predictor_score_18, predictor_bin_18)   as '18'
                       )
                   )
            where 1=1
                  and date_decision >= c_date_start
                  and date_decision <  c_date_end + 14
          ) scop
       on scop.skp_credit_case = scom.skp_credit_case
      and scop.date_decision = scom.date_decision
      and scop.skp_strategy_phase = scom.skp_strategy_phase
      and scop.pillar_name = scom.pillar_name
"""

In [9]:
connection_key = ['skp_credit_case','scorecard_name','pillar_name','predictor_name']

### Process data

In [10]:
connection_key = ['skp_credit_case','scorecard_name','pillar_name','predictor_name']
merge_statement = query_merge(df,table_name,schema,query_c_date,connection_key)
print(merge_statement)


MERGE INTO HCPH_RENNIELBAUTISTA.AH_RBB_POS_EC_SCORECARD_MONITORING DM1
USING
(

with tab_base as
(
select
       dm.id_cuid
      -- Basic Scoring
      ,case
            when dm.text_contract_number is not null
            then 1 else 0
       end                                                                      flag_with_app
      ,dm.text_contract_number
      ,dm.date_application
      ,dm.pure_population
      ,dm.pure_population_pre
      ,dm.prelim1_strategy_type
      ,dm.risk_grade_prelim1
      ,dm.flag_with_offer_v2_prelim1
      ,dm.flag_with_offer_v2
      -- DPOS Tags
      ,dm.flag_with_dpos_offer_prelim1
      ,dm.flag_with_dpos_offer
      ,dm.code_prm_decision
      ,case when bs.text_contract_number is not null then 1
            when dm.text_contract_number is not null then 0
       end                                                                      flag_dpos_app
      -- Basic Scoring Decision
      ,dm.cnt_approve                                          

## -----------------------------------------------------------------------------------------------------------

## Create PACKAGE

In [11]:
pckg_script = query_create_package(df,table_name,schema,query_c_date,connection_key)

### Package Head

In [12]:
pckg_head = pckg_script[0]
print(pckg_head)


CREATE OR REPLACE PACKAGE PCKG_AH_RBB_POS_EC_SCORECARD_MONITORING AS

    /****************************************************************************************************************************
    
        NAME:      PCKG_AH_RBB_POS_EC_SCORECARD_MONITORING
        PURPOSE:   Package to update AH_RBB_POS_EC_SCORECARD_MONITORING
        REVISIONS: 
        Ver        Date        Author                     Description
        ---------  ----------  ---------------            -----------------------------------------------------------------------
        1.0        2024-12-26  HCPH_RENNIELBAUTISTA       1. Created this package.
              
    ****************************************************************************************************************************/

    procedure proc_main (p_date_start date default trunc(sysdate-1), p_date_end date default trunc(sysdate));

END PCKG_AH_RBB_POS_EC_SCORECARD_MONITORING;



### Package Body

In [13]:
pckg_body = pckg_script[1]
print(pckg_body)


CREATE OR REPLACE PACKAGE BODY PCKG_AH_RBB_POS_EC_SCORECARD_MONITORING AS
    
/*----------------
------------------
----- proc00 -----
------------------
----------------*/
             
            
    procedure pstats( actable varchar2, anperc number default 0.01)
    is
    begin
        ap_public.core_log_pkg.pstart( 'Stat:'||actable );
        dbms_stats.gather_table_stats ( ownname => user, tabname => actable, estimate_percent => anperc );
        ap_public.core_log_pkg.pend;
    end pstats
    ;
    
/*----------------
------------------
----- proc01 -----
------------------
----------------*/
             
            
    procedure ptruncate( actable varchar2, anperc number default 0.01)
    is
    begin
        ap_public.core_log_pkg.pstart( 'trunc:'||actable );
        execute immediate 'truncate table '||actable ; 
        ap_public.core_log_pkg.pend ;
    end ptruncate
    ;
    
/*----------------
------------------
----- proc02 -----
------------------
---------------

# Query to SQL Table

This section shows different scenarios for creating SQL table using only select statement query and generates workbook if it does not exists. There are multiple checks in the defined function that requires user input to avoid accidentally overwriting existing SQL table.

## Scenario 1
No existing match for file name in local directory AND table name in the database.

### Pre-requisites

In [14]:
fname = 'TEST_DTM_CREATE.xlsx'
table_name = 'TEST_DTM_CREATE'
sheet_name = None

In [15]:
query = f'''
select * from ap_risk.summary_contract
where date_application = trunc(sysdate-3)
'''

In [16]:
query = apply_ora_hint(query)

### Process data

In [17]:
create_table_sql(query,fname,schema,table_name,sheet_name)
del fname
del table_name
del sheet_name
del query

Query used: 

-----------------------------------------------------------------

select /*+ materialize */ * from ap_risk.summary_contract
where date_application = trunc(sysdate-3)

-----------------------------------------------------------------

No matching file found.
Fetching Oracle data types.
Time elapsed: 0.148098
Fetching python data types.
Time elapsed: 28.654673
Converting to sqlalchemy data types.
Checking in the database.
Fetching Oracle data types.
Time elapsed: 0.470285
Fetching python data types.
Time elapsed: 20.915958
Converting to sqlalchemy data types.
Workbook created. Time elapsed: 41.037772 s
No existing table. Importing to database.
TEST_DTM_CREATE.xlsx
Table [TEST_DTM_CREATE] created.
Finished.
Execution time: 140.014214 s


## Scenario 2
No existing match for file name in local directory BUT table name exists in the database.

### Pre-requisites

In [18]:
fname = 'TEST_DTM_CREATE_2.xlsx'
table_name = 'TEST_DTM_CREATE'
sheet_name = None

In [19]:
query = f'''
select * from ap_risk.summary_contract
where date_application = trunc(sysdate-3)
'''

### Process data

In [20]:
create_table_sql(query,fname,schema,table_name,sheet_name)
del fname
del table_name
del sheet_name
del query

Query used: 

-----------------------------------------------------------------

select * from ap_risk.summary_contract
where date_application = trunc(sysdate-3)

-----------------------------------------------------------------

No matching file found.
Fetching Oracle data types.
Time elapsed: 0.161415
Fetching python data types.
Time elapsed: 25.704279
Converting to sqlalchemy data types.
Checking in the database.
Identical table with same table_name and column_names is found in the database. Do you want to create a new table? [YES/NO]  |  y
Insert new table name.  |  TEST_DTM_CREATE_2
New fullpath: C:\Users\Renniel.Bautista\Documents\Excel\TEST_DTM_CREATE_2.xlsx
Fetching Oracle data types.
Time elapsed: 0.471970
Fetching python data types.
Time elapsed: 23.807630
Converting to sqlalchemy data types.
Workbook created. Time elapsed: 47.879079 s
Importing to database.
Table [TEST_DTM_CREATE_2] created.
Finished.
Execution time: 330.069604 s


## Scenario 3
File already exists but no existing match in the database.

### Pre-requisites

In [21]:
fname = 'TEST_DTM_CREATE_2.xlsx'
table_name = 'TEST_DTM_CREATE_3'
sheet_name = None

In [22]:
query = f'''
select * from ap_risk.summary_contract
where date_application = trunc(sysdate-3)
'''

### Process data

In [23]:
create_table_sql(query,fname,schema,table_name,sheet_name)
del fname
del table_name
del sheet_name
del query

Query used: 

-----------------------------------------------------------------

select * from ap_risk.summary_contract
where date_application = trunc(sysdate-3)

-----------------------------------------------------------------

File already exists.
Fullpath: C:\Users\Renniel.Bautista\Documents\Excel\TEST_DTM_CREATE_2.xlsx
Fetching Oracle data types.
Time elapsed: 0.039864
Fetching python data types.
Time elapsed: 27.445258
Converting to sqlalchemy data types.
Checking in the database.
No existing table. Importing to database.
Table [TEST_DTM_CREATE_3] created.
Finished.
Execution time: 64.681479 s


## Scenario 4
Table name already exists in the database using same query.

### Pre-requisites

In [24]:
fname = 'TEST_DTM_CREATE_2.xlsx'
table_name = 'TEST_DTM_CREATE_3'
sheet_name = None

In [25]:
query = f'''
select * from ap_risk.summary_contract
where date_application = trunc(sysdate-3)
'''

### Process data

In [26]:
create_table_sql(query,fname,schema,table_name,sheet_name)
del fname
del table_name
del sheet_name
del query

Query used: 

-----------------------------------------------------------------

select * from ap_risk.summary_contract
where date_application = trunc(sysdate-3)

-----------------------------------------------------------------

File already exists.
Fullpath: C:\Users\Renniel.Bautista\Documents\Excel\TEST_DTM_CREATE_2.xlsx
Fetching Oracle data types.
Time elapsed: 0.050687
Fetching python data types.
Time elapsed: 30.926036
Converting to sqlalchemy data types.
Checking in the database.
Table already exists in the database. Do you want to create a new table? [YES/NO]  |  y
Insert new table name.  |  TEST_DTM_CREATE_4
New fullpath: C:\Users\Renniel.Bautista\Documents\Excel\TEST_DTM_CREATE_4.xlsx
Fetching Oracle data types.
Time elapsed: 0.418957
Fetching python data types.
Time elapsed: 23.185066
Converting to sqlalchemy data types.
Workbook created. Time elapsed: 55.336544 s
Importing to database.
Table [TEST_DTM_CREATE_4] created.
Finished.
Execution time: 268.691036 s


## Scenario 5
Table name already exists in the database using different query.

### Pre-requisites

In [27]:
fname = 'TEST_DTM_CREATE_2.xlsx'
table_name = 'TEST_DTM_CREATE_3'
sheet_name = None

In [28]:
query = f'''
select * from ap_risk.pos_ec_datamart_tree
where date_application = trunc(sysdate-3)
'''

### Process data

In [29]:
create_table_sql(query,fname,schema,table_name,sheet_name)
del fname
del table_name
del sheet_name
del query

Query used: 

-----------------------------------------------------------------

select * from ap_risk.pos_ec_datamart_tree
where date_application = trunc(sysdate-3)

-----------------------------------------------------------------

File already exists.
Fullpath: C:\Users\Renniel.Bautista\Documents\Excel\TEST_DTM_CREATE_2.xlsx
Fetching Oracle data types.
Time elapsed: 0.140335
Fetching python data types.
Time elapsed: 20.560579
Converting to sqlalchemy data types.
Checking in the database.
Table already exists in the database. Do you want to create a new table? [YES/NO]  |  y
Insert new table name.  |  TEST_DTM_CREATE_5
New fullpath: C:\Users\Renniel.Bautista\Documents\Excel\TEST_DTM_CREATE_5.xlsx
Fetching Oracle data types.
Time elapsed: 0.463542
Fetching python data types.
Time elapsed: 9.981015
Converting to sqlalchemy data types.
Workbook created. Time elapsed: 14.643381 s
Importing to database.
Table [TEST_DTM_CREATE_5] created.
Finished.
Execution time: 89.563301 s


# Query to Excel
This section display the function query_to_excel that generates an Excel file on a pre-destined fullpath using SQLquery. It involves pre-processing information from database to enable backtracing and avoid XML errors when saving to workbook. When saving large dataset exceeding Excel maximum rows, it will automatically chunk the data into different sheets.

## Pre-requisites

In [30]:
fname = 'TEST_QUERY_TO_EXCEL.xlsx'
table_name = 'TEST_QUERY_TO_EXCEL'
sheet_name = None

In [31]:
query = f'''
select * from ap_risk.summary_contract
where date_application = trunc(sysdate-3)
'''

In [32]:
# Apply Oracle hint in the query if none. This step is optional.
query = apply_ora_hint(query)

## Process data

In [33]:
query_to_excel(schema,query,fname,sheet_name)
del fname
del table_name
del sheet_name
del query

CHUNK 1 appended to list. Time elapsed: 14.246343
Chunking done. Time elapsed: 14.312488 s
Fetching Oracle data types.
Time elapsed: 0.328385
Fetching python data types.
Time elapsed: 23.194575
Converting to sqlalchemy data types.
Dataframe created. Time elapsed: 41.715835 s
Workbook created. Time elapsed: 46.021630 s
Execution time: 102.061356 s


# Query to Data Frame
This sections generates a data frame using query. It involves pre-processing information from database to enable backtracing. When dealing with large dataset, chunksize parameter can be set before reading from database to allow faster execution.

## Pre-requisites

In [34]:
query = f'''
select * from ap_risk.summary_contract
where date_application = trunc(sysdate-3)
'''

In [35]:
# Apply Oracle hint in the query if none. This step is optional.
query = apply_ora_hint(query)

## Process data

In [36]:
df = query_to_df(query,schema)
del query

df

CHUNK 1 appended to list. Time elapsed: 9.006680
Chunking done. Time elapsed: 9.068983 s
Fetching Oracle data types.
Time elapsed: 0.548904
Fetching python data types.
Time elapsed: 7.217293
Converting to sqlalchemy data types.
Chunk list deleted.
Dataframe created. Time elapsed: 13.633889 s
Execution time: 22.702872 s


,SKP_CREDIT_CASE,TEXT_CONTRACT_NUMBER,ID_CUID,NAME_CREDIT_STATUS,MS,DATE_APPLICATION,WEEK_APPLICATION,MONTH_APPLICATION,HOUR_APPLICATION,FLAG_LAST7DAYS,FLAG_LAST7_WEEKS,FLAG_LAST_7_MONTHS,FLAG_WORKING_DAY,DTIME_SIGN,DTIME_ACTIVE,DTIME_RETURN,CODE_GENDER,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,CODE_HOUSING_TYPE,CNT_PERSON_DEPENDENT,DATE_BIRTH,AGE,DATE_EMPLOYED_FROM,NAME_EMPLOYER,NAME_INCOME_TYPE,CODE_PROFESSION,CODE_EMPLOYER_INDUSTRY,AMT_INCOME_MAIN,INCOME_CATEGORY,AMT_INCOME_OTHER,AMT_REMITTANCE,AMT_EXPENSE_DEBT,PARTNER,CODE_SELLER,CODE_SELLER_CATEGORY,NAME_SALESROOM,MALL,CODE_SALESROOM,NAME_EMPL,DSM,NAME_GOODS_CATEGORY,NAME_GOODS_TYPE,NAME_PRODUCER,TEXT_MODEL_NUMBER,AMT_CREDIT,AMT_GOODS_PRICE,AMT_BUNDLES_MAX,AMT_DOWN_PAYMENT,AMT_ANNUITY,AMT_FEE_ORIGINATION,AMT_PAYMENT_CASH,CNT_INSTALMENT,RATE_FLAT,RATE_INTEREST,HC_RATE_OLD,HC_RATE,HC_RATE_W_GIFT_ADJ,FA_CATEGORY,PRICE_CATEGORY,DP_CATEGORY,TERM_CATEGORY,AVG_CASH,CNT_APPROVE,CNT_REJECTED,CNT_IN_PROCESS,CNT_SIGNED,CNT_CANCELLED_IN_PROCESS,CNT_RETURNED,CNT_ACTIVE,CNT_CANCELLED_POST,FLAG_CANCELLED_PRE_APPROVAL,CODE_EMPLOYEE,WEIGHTED_TERM,WEIGHTED_HC_RATE,DURATION,REMITTANCE_TYPE,FINGER_PRINT_RECIEVED,FLAG_INSURANCE,FLAG_CASHLOAN,FLAG_ACTION,FLAG_GIFT,FLAG_BUNDLES,CNT_PRODUCT,POS_REGION,POS_DISTRICT,CODE_PRODUCT,REMITTER,UWI_REGION,SKP_CLIENT,SKP_PRODUCT,SKP_CONTRACT,SKP_APPLICATION,SKP_CREDIT_TYPE,FLAG_KNOWN_APPLICANT,FLAG_KNOWN_CLIENT,FLAG_TOP_UP,NAME_FIRST,NAME_LAST,NAME_MIDDLE,NAME_MOTHER,FLAG_ONLINE_MCL,PRODUCT_TYPE,FLAG_MC_ELIGIBLE,FLAG_ESIGN,POS_GROUP,FLAG_SOLD,FLAG_POSITIVE_OFFER,ORIGINAL_AMT_PAYMENT_CASH,FLAG_EXISTING_SC,FLAG_MOB_APP,DATE_DECISION,FLAG_WORKING_DAY_DECISION,RATE_PAYMENT_CASH_MIN,EXTRA_DP_RATE,EXTRA_DP_AMOUNT,NAME_CLASS_STATUS,ANON_SKP_CREDIT_CASE,AMT_CREDIT_NO_VAS,RISK_GRADE_CBN_POS,AMT_ANNUITY_WO_FEE,CANCELLATION_TYPE,TEXT_CANCELLATION_REASON,RG_PD,RG_SIMULATED,COLUMN1,COLUMN2,FLAG_RBP,FLAG_AFTER_RBP_ROLL_OUT,FLAG_RBP_WAVE_DATE,FLAG_RBP_WAVE_AREA,SNAPP_LOAN_APP_NO,LIN_SCORE_UNIVERSAL,PD_SCORE_UNIVERSAL,UNIVERSAL_RISK_GRADE,FIRST_BOD_CHANNEL,SECOND_BOD_CHANNEL,LIN_SCORE_UNIVERSAL_OLD,PD_SCORE_UNIVERSAL_OLD,UNIVERSAL_RISK_GRADE_OLD,FLAG_GUARANTEED_LIMIT,ID_CUID_UPDATED,SKP_CLIENT_UPDATED,REJECT_REASON,REJECTION_STAGE,REJECT_REASON_OWNER,REJECT_REASON_TOTAL,FLAG_PILOT_BVM,REL_RG,PRODUCT_TYPE_V2,PRERESULT_FANT_TYPE,PRERESULT_FANT_CODE,ATTRIBUTION_TYPE
0,7617760262,103512623297,4616875356,Rejected,Other,2025-01-23 00:00:00,2025-01-20 00:00:00,2025-01-01 00:00:00,11,1,1,1,1,nan,nan,nan,M,XNA,Single,LIVING_WITH_PARENTS,0.0,2000-08-10 00:00:00,24.0,2021-06-01 00:00:00,converge,Private/Employed,OTHER,OTHER,15000.0,IM_10+_17,0.0,NaN,0.0,#Mobile Paperless,01429,STAND,Mobile Paperless,NaN,001270009,Mobile_User,NaN,Mobile / Tablet,Mobile Phone,VIVO,Vivo V40 Lite 5G 8+256,*,*,*,*,*,*,*,12.0,*,*,*,*,*,FA_15+_20,GP_15+_20,DP_0%_25%,Term_0_12,0.200013,0,1,0,0,0,0,0,0,N,Mobile_User,196944.0,0.0,NaN,No remittance,NaN,1,0,1,0,0,1.0,OTHER,LA UNION,LZ00MPVI5B,nan,I (ILOCOS REGION),6643399071,3035912,-79,14753948741,1,0,0,0,Julius Grand R.K.,*,Supsup,*,0,POS,NaN,0,XNA,NaN,0.0,3200.0,0,1,2025-01-23 00:00:00,1,0.0,0.200013,3200.0,Acquisition,2c684d89-aa59-7bb2-e063-2206480ac1e0,12799.0,E,1368.0,nan,nan,H,K,NaN,NaN,1.0,NaN,nan,WAVE_4,4500548839,-1.959957,0.123472,K2,MOB,SNAPP,NaN,NaN,NaN,0.0,4616875356,6643399071,SCO,PRE,UWI,1.0,NaN,NaN,POS,nan,nan,nan
1,7617760479,103512865510,330313354,Active,Other,2025-01-23 00:00:00,2025-01-20 00:00:00,2025-01-01 00:00:00,16,1,1,1,1,2025-01-23 16:41:32,2025-01-23 16:48:42,nan,M,XNA,Single,LIVING_WITH_RELATIVES,0.0,1996-09-22 00:00:00,28.0,2022-03-01 00:00:00,Vitoviel Par,Private/Employed,OTHER,TRADE,28512.0,IM_25+_40,0.0,NaN,0.0,LGK Guanzon Incorporated,00353,RKA,Guanzon Mobitek - Nepo Mall Alaminos,NaN,003530004,Mobile_User,NaN,Mobile / Tablet,Mobile Phone,Xiaomi,Xiaomi 14T (12+512),*,*,*,*,*,*,*,12.0,*,*,*,*,*,FA_20+_30,GP_20+_30,DP_0%_25%,Term_0_12,0.000000,1,0,0,1,0,0,1,0,N,Mobile_User,355296.0,0.0,NaN,No remittance,NaN,1,0,1,0,0,1.0,LUZON_NO

# Variable Query to Data Frame
This section utilizes SQL query with date parameters. When dealing with large dataset and chunking method is insufficient, this function will load the SQL query on separate dates thus partitioning it to multiple small datasets before concatenating into one dataframe.

## Pre-requisites

In [37]:
query_c_date = f'''
select * from ap_risk.summary_contract
where 1=1
      and date_application >= c_date_start
      and date_application <  c_date_end
'''

c_date_start = "DATE '2024-10-07'"
c_date_end = "DATE '2024-10-14'"

# ## Optional arguments
# time_out = 0 # default = 60
# temporal = 'weekly' # default = 'monthly'
# chunk_size = 500000 # default = 1000000

In [38]:
# Apply Oracle hint in the query if none. This step is optional.
query_c_date = apply_ora_hint(query_c_date)

## Process data

In [39]:
# df = var_query_to_df(query_c_date,schema,c_date_start,c_date_end,time_out,temporal,chunk_size=chunk_size)
df = var_query_to_df(query_c_date,schema,c_date_start,c_date_end)
del query_c_date
del c_date_start
del c_date_end

df

Timeline: DATE '2024-10-07' to DATE '2024-10-14'
Process takes too long. Partitioning timeline by day and populating individually.
Timeline: DATE '2024-10-07' to DATE '2024-10-08'
CHUNK 1 appended to list. Time elapsed: 8.452637
Chunking done. Time elapsed: 8.526656 s
Fetching Oracle data types.
Time elapsed: 0.502067
Fetching python data types.
Time elapsed: 7.935249
Converting to sqlalchemy data types.
Chunk list deleted.
Partial Dataframe 1 created. Time elapsed: 13.364444 s
Appended to main dataframe. Time elapsed: 82.249285 s
Timeline: DATE '2024-10-08' to DATE '2024-10-09'
CHUNK 1 appended to list. Time elapsed: 8.760125
Chunking done. Time elapsed: 8.814749 s
Fetching Oracle data types.
Time elapsed: 0.512429
Fetching python data types.
Time elapsed: 7.966558
Converting to sqlalchemy data types.
Chunk list deleted.
Partial Dataframe 2 created. Time elapsed: 13.203205 s
Appended to main dataframe. Time elapsed: 104.273894 s
Timeline: DATE '2024-10-09' to DATE '2024-10-10'
CHUNK 1

,SKP_CREDIT_CASE,TEXT_CONTRACT_NUMBER,ID_CUID,NAME_CREDIT_STATUS,MS,DATE_APPLICATION,WEEK_APPLICATION,MONTH_APPLICATION,HOUR_APPLICATION,FLAG_LAST7DAYS,FLAG_LAST7_WEEKS,FLAG_LAST_7_MONTHS,FLAG_WORKING_DAY,DTIME_SIGN,DTIME_ACTIVE,DTIME_RETURN,CODE_GENDER,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,CODE_HOUSING_TYPE,CNT_PERSON_DEPENDENT,DATE_BIRTH,AGE,DATE_EMPLOYED_FROM,NAME_EMPLOYER,NAME_INCOME_TYPE,CODE_PROFESSION,CODE_EMPLOYER_INDUSTRY,AMT_INCOME_MAIN,INCOME_CATEGORY,AMT_INCOME_OTHER,AMT_REMITTANCE,AMT_EXPENSE_DEBT,PARTNER,CODE_SELLER,CODE_SELLER_CATEGORY,NAME_SALESROOM,MALL,CODE_SALESROOM,NAME_EMPL,DSM,NAME_GOODS_CATEGORY,NAME_GOODS_TYPE,NAME_PRODUCER,TEXT_MODEL_NUMBER,AMT_CREDIT,AMT_GOODS_PRICE,AMT_BUNDLES_MAX,AMT_DOWN_PAYMENT,AMT_ANNUITY,AMT_FEE_ORIGINATION,AMT_PAYMENT_CASH,CNT_INSTALMENT,RATE_FLAT,RATE_INTEREST,HC_RATE_OLD,HC_RATE,HC_RATE_W_GIFT_ADJ,FA_CATEGORY,PRICE_CATEGORY,DP_CATEGORY,TERM_CATEGORY,AVG_CASH,CNT_APPROVE,CNT_REJECTED,CNT_IN_PROCESS,CNT_SIGNED,CNT_CANCELLED_IN_PROCESS,CNT_RETURNED,CNT_ACTIVE,CNT_CANCELLED_POST,FLAG_CANCELLED_PRE_APPROVAL,CODE_EMPLOYEE,WEIGHTED_TERM,WEIGHTED_HC_RATE,DURATION,REMITTANCE_TYPE,FINGER_PRINT_RECIEVED,FLAG_INSURANCE,FLAG_CASHLOAN,FLAG_ACTION,FLAG_GIFT,FLAG_BUNDLES,CNT_PRODUCT,POS_REGION,POS_DISTRICT,CODE_PRODUCT,REMITTER,UWI_REGION,SKP_CLIENT,SKP_PRODUCT,SKP_CONTRACT,SKP_APPLICATION,SKP_CREDIT_TYPE,FLAG_KNOWN_APPLICANT,FLAG_KNOWN_CLIENT,FLAG_TOP_UP,NAME_FIRST,NAME_LAST,NAME_MIDDLE,NAME_MOTHER,FLAG_ONLINE_MCL,PRODUCT_TYPE,FLAG_MC_ELIGIBLE,FLAG_ESIGN,POS_GROUP,FLAG_SOLD,FLAG_POSITIVE_OFFER,ORIGINAL_AMT_PAYMENT_CASH,FLAG_EXISTING_SC,FLAG_MOB_APP,DATE_DECISION,FLAG_WORKING_DAY_DECISION,RATE_PAYMENT_CASH_MIN,EXTRA_DP_RATE,EXTRA_DP_AMOUNT,NAME_CLASS_STATUS,ANON_SKP_CREDIT_CASE,AMT_CREDIT_NO_VAS,RISK_GRADE_CBN_POS,AMT_ANNUITY_WO_FEE,CANCELLATION_TYPE,TEXT_CANCELLATION_REASON,RG_PD,RG_SIMULATED,COLUMN1,COLUMN2,FLAG_RBP,FLAG_AFTER_RBP_ROLL_OUT,FLAG_RBP_WAVE_DATE,FLAG_RBP_WAVE_AREA,SNAPP_LOAN_APP_NO,LIN_SCORE_UNIVERSAL,PD_SCORE_UNIVERSAL,UNIVERSAL_RISK_GRADE,FIRST_BOD_CHANNEL,SECOND_BOD_CHANNEL,LIN_SCORE_UNIVERSAL_OLD,PD_SCORE_UNIVERSAL_OLD,UNIVERSAL_RISK_GRADE_OLD,FLAG_GUARANTEED_LIMIT,ID_CUID_UPDATED,SKP_CLIENT_UPDATED,REJECT_REASON,REJECTION_STAGE,REJECT_REASON_OWNER,REJECT_REASON_TOTAL,FLAG_PILOT_BVM,REL_RG,PRODUCT_TYPE_V2,PRERESULT_FANT_TYPE,PRERESULT_FANT_CODE,ATTRIBUTION_TYPE
0,7377902009,101338567157,3939666346,Cancelled,Other,2024-10-07 00:00:00,2024-10-07 00:00:00,2024-10-01 00:00:00,0,0,0,1,1,nan,nan,nan,F,XNA,Single,LIVING_WITH_PARENTS,7.0,2002-07-16 00:00:00,22.0,2022-10-06 00:00:00,online seller,Self-employed,XNA,OTHER,15000.0,IM_10+_17,0.0,NaN,0.0,#Mobile Paperless,01429,STAND,GMA Virtual Salesroom,NaN,THYGSTU5I,Koyal_User,NaN,nan,nan,nan,nan,*,*,*,*,*,*,*,NaN,*,*,*,*,*,XNA,XNA,XNA,XNA,NaN,0,0,0,0,1,0,0,0,Y,Koyal_User,NaN,NaN,NaN,No remittance,NaN,0,0,0,0,0,NaN,OTHER,MANILA CITY,XNA,nan,NCR_1,6284148727,-1,-79,14451138957,-1,0,0,0,ressan,*,Bejarin,*,0,POS,NaN,0,XNA,NaN,NaN,NaN,0,1,2024-10-21 00:00:00,1,-1.0,NaN,NaN,Acquisition,2505d5b3-b8e1-bfc6-e063-2206480a44e8,NaN,J,NaN,Scheduler,XSELL_NOT_TAKEN,N/A,K,NaN,NaN,0.0,NaN,nan,NON-RBP,nan,NaN,NaN,nan,GMA,nan,NaN,NaN,NaN,0.0,3939666346,6284148727,nan,nan,nan,NaN,NaN,NaN,POS,NaN,NaN,nan
1,7377805289,101338567295,3462652868,Cancelled,Other,2024-10-07 00:00:00,2024-10-07 00:00:00,2024-10-01 00:00:00,0,0,0,1,1,nan,nan,nan,M,XNA,Single,OWNED,0.0,1990-02-20 00:00:00,34.0,2017-07-06 00:00:00,Rj meatshop/ argie sari sari storw,Self-employed,XNA,FOOD_BEVERAGE,3532700.0,IM_40+,0.0,NaN,0.0,#Mobile Paperless,01429,STAND,GMA Virtual Salesroom,NaN,THYGSTU5I,Koyal_User,NaN,nan,nan,nan,nan,*,*,*,*,*,*,*,NaN,*,*,*,*,*,XNA,XNA,XNA,XNA,NaN,0,0,0,0,1,0,0,0,Y,Koyal_User,NaN,NaN,NaN,No remittance,NaN,0,0,0,0,0,NaN,OTHER,MANILA CITY,XNA,nan,NCR_1,6033868828,-1,-79,14416636832,-1,0,0,0,ARGIE,*,SAMUYA,*,0,POS,NaN,0,XNA,NaN,NaN,NaN,0,1,2024-10-09 00:00:00,1,-1.0,NaN,NaN,Acquisition,2413e85c-17ab-b968-e063-2206480ac480,NaN,E,NaN,Employee,NO_MORE_INTEREST,N/A,

# Data Frame to Excel
This section pre-process the data to avoid XML errors when saving to workbook. When saving large dataset exceeding Excel maximum rows, it will automatically chunk the data into different sheets.

## Pre-requisites

In [40]:
fname = 'TEST_DF_TO_EXCEL.xlsx'

### Create sample data frame with 10M x 5 shape to test performance on large dataset.

In [41]:
# Generate data for each column
column_1 = np.random.randint(10_000_000, 11_000_000, size=(10_000_000)) # Integers
column_2 = np.random.randint(490_000, 800_000, size=(10_000_000))/100 # Floats
column_3 = np.random.choice(['A+','A', 'B', 'C', 'D','E','F','G','H','I','J','K'], size=(10_000_000)) # Categorical
column_4 = pd.date_range('2023-01-01', periods=10_000_000, freq='T') # Datetime
column_5 = np.random.randint(490_000*6, 12_000_000, size=(10_000_000))/100 # # Another set of Floats

# Create DataFrame
data = {
    'ID_CUID': column_1,
    'AMT_ANNUITY': column_2,
    'RISK_GRADE': column_3,
    'DATE_APPLICATION': column_4,
    'AMT_CREDIT': column_5
}

df = pd.DataFrame(data)
df

,ID_CUID,AMT_ANNUITY,RISK_GRADE,DATE_APPLICATION,AMT_CREDIT
0,10211944,6286.87,B,2023-01-01 00:00:00,98564.27
1,10353989,6177.47,G,2023-01-01 00:01:00,46433.40
2,10001180,5670.16,I,2023-01-01 00:02:00,75153.46
3,10098704,6821.06,E,2023-01-01 00:03:00,92525.11
4,10607189,6733.63,D,2023-01-01 00:04:00,29932.05
...,...,...,...,...,...
9999995,10503234,5661.04,A+,2042-01-05 10:35:00,49988.77
9999996,10012174,7894.52,C,2042-01-05 10:36:00,36296.22
9999997,10176573,6644.12,B,2042-01-05 10:37:00,85004.93
9999998,10235586,4933.73,G,2042-01-05 10:38:00,69956.30


## Process data

In [42]:
df_to_excel(df,schema,fname)
del fname
del column_1
del column_2
del column_3
del column_4
del column_5
del data
del df

Fetching Oracle data types.
Time elapsed: 0.685464
Fetching python data types.
Time elapsed: 348.090451
Converting to sqlalchemy data types.
New sheet created for CHUNK 1. Time elapsed: 8.592081
New sheet created for CHUNK 2. Time elapsed: 9.264383
New sheet created for CHUNK 3. Time elapsed: 8.299145
New sheet created for CHUNK 4. Time elapsed: 8.198022
New sheet created for CHUNK 5. Time elapsed: 8.583251
New sheet created for CHUNK 6. Time elapsed: 8.112489
New sheet created for CHUNK 7. Time elapsed: 8.296040
New sheet created for CHUNK 8. Time elapsed: 8.489466
New sheet created for CHUNK 9. Time elapsed: 8.681748
New sheet created for CHUNK 10. Time elapsed: 8.462907
New sheet created for CHUNK 11. Time elapsed: 0.059578
Workbook created. Time elapsed: 550.202152 s


# Excel to SQL
This section reads the excel file, parse datetime columns in the correct format, convert python types into SQLalchemy types suitable for Oracle database before importing to SQL as table.

## Pre-requisites

In [43]:
fname = 'TEST_QUERY_TO_EXCEL.xlsx'
table_name = 'TEST_EXCEL_TO_SQL'
sheet_name = 'Sheet1'

## Process data

In [44]:
excel_to_sql(fname,schema,table_name,sheet_name)
del fname
del table_name
del sheet_name

Reading file as dataframe.
Time elapsed: 12.031705
Pre-processing data types.
Fetching Oracle data types.
Time elapsed: 0.306824
Fetching python data types.
Time elapsed: 20.099096
Converting to sqlalchemy data types.
Time elapsed: 33.865754
Fixing string datetime.
Time elapsed: 19.584854
Importing to database.
Execution time: 77.531092 s


# Insert Into SQL Table (per row)

## Pre-requisites

In [45]:
table_name = 'TEST_DTM_CREATE'
source_owner = 'AP_RISK'
source_table = 'SUMMARY_CONTRACT'
condition = "date_application >= trunc(sysdate-2)"

query_base = f'''
select * from {schema}.{table_name}
'''
query = f'''
select * from {source_owner}.{source_table}
{sql_where_condition(condition)}
'''

df_base = query_to_df(query_base,schema)
display(df_base.head())

df = query_to_df(query,schema)
display(df.head())

CHUNK 1 appended to list. Time elapsed: 28.169580
Chunking done. Time elapsed: 28.293394 s
Fetching Oracle data types.
Time elapsed: 0.984658
Fetching python data types.
Time elapsed: 6.634466
Converting to sqlalchemy data types.
Chunk list deleted.
Dataframe created. Time elapsed: 15.069928 s
Execution time: 43.363322 s


,SKP_CREDIT_CASE,TEXT_CONTRACT_NUMBER,ID_CUID,NAME_CREDIT_STATUS,MS,DATE_APPLICATION,WEEK_APPLICATION,MONTH_APPLICATION,HOUR_APPLICATION,FLAG_LAST7DAYS,FLAG_LAST7_WEEKS,FLAG_LAST_7_MONTHS,FLAG_WORKING_DAY,DTIME_SIGN,DTIME_ACTIVE,DTIME_RETURN,CODE_GENDER,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,CODE_HOUSING_TYPE,CNT_PERSON_DEPENDENT,DATE_BIRTH,AGE,DATE_EMPLOYED_FROM,NAME_EMPLOYER,NAME_INCOME_TYPE,CODE_PROFESSION,CODE_EMPLOYER_INDUSTRY,AMT_INCOME_MAIN,INCOME_CATEGORY,AMT_INCOME_OTHER,AMT_REMITTANCE,AMT_EXPENSE_DEBT,PARTNER,CODE_SELLER,CODE_SELLER_CATEGORY,NAME_SALESROOM,MALL,CODE_SALESROOM,NAME_EMPL,DSM,NAME_GOODS_CATEGORY,NAME_GOODS_TYPE,NAME_PRODUCER,TEXT_MODEL_NUMBER,AMT_CREDIT,AMT_GOODS_PRICE,AMT_BUNDLES_MAX,AMT_DOWN_PAYMENT,AMT_ANNUITY,AMT_FEE_ORIGINATION,AMT_PAYMENT_CASH,CNT_INSTALMENT,RATE_FLAT,RATE_INTEREST,HC_RATE_OLD,HC_RATE,HC_RATE_W_GIFT_ADJ,FA_CATEGORY,PRICE_CATEGORY,DP_CATEGORY,TERM_CATEGORY,AVG_CASH,CNT_APPROVE,CNT_REJECTED,CNT_IN_PROCESS,CNT_SIGNED,CNT_CANCELLED_IN_PROCESS,CNT_RETURNED,CNT_ACTIVE,CNT_CANCELLED_POST,FLAG_CANCELLED_PRE_APPROVAL,CODE_EMPLOYEE,WEIGHTED_TERM,WEIGHTED_HC_RATE,DURATION,REMITTANCE_TYPE,FINGER_PRINT_RECIEVED,FLAG_INSURANCE,FLAG_CASHLOAN,FLAG_ACTION,FLAG_GIFT,FLAG_BUNDLES,CNT_PRODUCT,POS_REGION,POS_DISTRICT,CODE_PRODUCT,REMITTER,UWI_REGION,SKP_CLIENT,SKP_PRODUCT,SKP_CONTRACT,SKP_APPLICATION,SKP_CREDIT_TYPE,FLAG_KNOWN_APPLICANT,FLAG_KNOWN_CLIENT,FLAG_TOP_UP,NAME_FIRST,NAME_LAST,NAME_MIDDLE,NAME_MOTHER,FLAG_ONLINE_MCL,PRODUCT_TYPE,FLAG_MC_ELIGIBLE,FLAG_ESIGN,POS_GROUP,FLAG_SOLD,FLAG_POSITIVE_OFFER,ORIGINAL_AMT_PAYMENT_CASH,FLAG_EXISTING_SC,FLAG_MOB_APP,DATE_DECISION,FLAG_WORKING_DAY_DECISION,RATE_PAYMENT_CASH_MIN,EXTRA_DP_RATE,EXTRA_DP_AMOUNT,NAME_CLASS_STATUS,ANON_SKP_CREDIT_CASE,AMT_CREDIT_NO_VAS,RISK_GRADE_CBN_POS,AMT_ANNUITY_WO_FEE,CANCELLATION_TYPE,TEXT_CANCELLATION_REASON,RG_PD,RG_SIMULATED,COLUMN1,COLUMN2,FLAG_RBP,FLAG_AFTER_RBP_ROLL_OUT,FLAG_RBP_WAVE_DATE,FLAG_RBP_WAVE_AREA,SNAPP_LOAN_APP_NO,LIN_SCORE_UNIVERSAL,PD_SCORE_UNIVERSAL,UNIVERSAL_RISK_GRADE,FIRST_BOD_CHANNEL,SECOND_BOD_CHANNEL,LIN_SCORE_UNIVERSAL_OLD,PD_SCORE_UNIVERSAL_OLD,UNIVERSAL_RISK_GRADE_OLD,FLAG_GUARANTEED_LIMIT,ID_CUID_UPDATED,SKP_CLIENT_UPDATED,REJECT_REASON,REJECTION_STAGE,REJECT_REASON_OWNER,REJECT_REASON_TOTAL,FLAG_PILOT_BVM,REL_RG
0,7549734606,101389261549,4418696297,In Preprocess,Other,2024-12-23 00:00:00,2024-12-23 00:00:00,2024-12-01 00:00:00,10,1,1,1,1,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,NaN,NaN,NaN,#Mobile Paperless,01429,STAND,GMA Virtual Salesroom,NaN,THYGSTU5I,nan,NaN,nan,nan,nan,nan,*,*,*,*,*,*,*,NaN,*,*,*,*,*,XNA,XNA,XNA,XNA,NaN,0,0,1,0,0,0,0,0,N,nan,NaN,NaN,NaN,No remittance,NaN,0,0,0,0,0,NaN,OTHER,nan,XNA,nan,nan,6545716277,-1,-79,-43,-1,0,0,0,nan,*,nan,*,0,POS,NaN,0,XNA,NaN,NaN,NaN,0,1,nan,1,NaN,NaN,NaN,nan,2a20abb2-06cb-8809-e063-2206480a7361,NaN,nan,NaN,nan,nan,N/A,nan,NaN,NaN,0.0,NaN,NON-RBP,NON-RBP,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,4418696297,6545716277,nan,nan,nan,NaN,NaN,NaN
1,7549734699,101389564528,1897626316,Cancelled,Other,2024-12-23 00:00:00,2024-12-23 00:00:00,2024-12-01 00:00:00,13,1,1,1,1,nan,nan,nan,F,XNA,Single,LIVING_WITH_PARENTS,0.0,2000-12-30 00:00:00,23.0,2021-12-01 00:00:00,Sheryll Barbo Esponilla,Self-employed,OTHER,OTHER,45000.0,IM_40+,0.0,NaN,0.0,#Mobile Paperless,01429,STAND,GMA Virtual Salesroom,NaN,THYGSTU5I,Koyal_User,NaN,nan,nan,nan,nan,*,*,*,*,*,*,*,NaN,*,*,*,*,*,XNA,XNA,XNA,XNA,NaN,0,0,0,0,1,0,0,0,Y,Koyal_User,NaN,NaN,NaN,No remittance,NaN,0,0,0,0,0,NaN,OTHER,MANILA CITY,XNA,nan,NCR_1,4612641593,-1,-79,14657633385,-1,0,1,0,Sheryll,*,Barbo,*,0,POS,NaN,0,XNA,NaN,NaN,NaN,1,1,2024-12-23 00:00:00,1,-1.0,NaN,NaN,Non Acquisition,29f7fad3-16aa-caaf-e063-2206480abb04,0.0,nan,NaN,Employee,NO_MORE_INTEREST,N/A,nan,NaN,NaN,0.0,NaN,NON-RBP,NON-RBP,NaN,NaN,NaN,nan,GMA,nan,NaN,NaN,NaN,0.0,1897626316,4612641593,nan,nan,nan,NaN,NaN,NaN
2,7549734947,101389191606,3095400064,In Preprocess,Other,2024-12-23 00:00:00,2024-12-23 00:00:00,2024-12-01 00:00:0

CHUNK 1 appended to list. Time elapsed: 13.392697
Chunking done. Time elapsed: 13.485481 s
Fetching Oracle data types.
Time elapsed: 0.539205
Fetching python data types.
Time elapsed: 6.594281
Converting to sqlalchemy data types.
Chunk list deleted.
Dataframe created. Time elapsed: 13.391820 s
Execution time: 26.878317 s


,SKP_CREDIT_CASE,TEXT_CONTRACT_NUMBER,ID_CUID,NAME_CREDIT_STATUS,MS,DATE_APPLICATION,WEEK_APPLICATION,MONTH_APPLICATION,HOUR_APPLICATION,FLAG_LAST7DAYS,FLAG_LAST7_WEEKS,FLAG_LAST_7_MONTHS,FLAG_WORKING_DAY,DTIME_SIGN,DTIME_ACTIVE,DTIME_RETURN,CODE_GENDER,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,CODE_HOUSING_TYPE,CNT_PERSON_DEPENDENT,DATE_BIRTH,AGE,DATE_EMPLOYED_FROM,NAME_EMPLOYER,NAME_INCOME_TYPE,CODE_PROFESSION,CODE_EMPLOYER_INDUSTRY,AMT_INCOME_MAIN,INCOME_CATEGORY,AMT_INCOME_OTHER,AMT_REMITTANCE,AMT_EXPENSE_DEBT,PARTNER,CODE_SELLER,CODE_SELLER_CATEGORY,NAME_SALESROOM,MALL,CODE_SALESROOM,NAME_EMPL,DSM,NAME_GOODS_CATEGORY,NAME_GOODS_TYPE,NAME_PRODUCER,TEXT_MODEL_NUMBER,AMT_CREDIT,AMT_GOODS_PRICE,AMT_BUNDLES_MAX,AMT_DOWN_PAYMENT,AMT_ANNUITY,AMT_FEE_ORIGINATION,AMT_PAYMENT_CASH,CNT_INSTALMENT,RATE_FLAT,RATE_INTEREST,HC_RATE_OLD,HC_RATE,HC_RATE_W_GIFT_ADJ,FA_CATEGORY,PRICE_CATEGORY,DP_CATEGORY,TERM_CATEGORY,AVG_CASH,CNT_APPROVE,CNT_REJECTED,CNT_IN_PROCESS,CNT_SIGNED,CNT_CANCELLED_IN_PROCESS,CNT_RETURNED,CNT_ACTIVE,CNT_CANCELLED_POST,FLAG_CANCELLED_PRE_APPROVAL,CODE_EMPLOYEE,WEIGHTED_TERM,WEIGHTED_HC_RATE,DURATION,REMITTANCE_TYPE,FINGER_PRINT_RECIEVED,FLAG_INSURANCE,FLAG_CASHLOAN,FLAG_ACTION,FLAG_GIFT,FLAG_BUNDLES,CNT_PRODUCT,POS_REGION,POS_DISTRICT,CODE_PRODUCT,REMITTER,UWI_REGION,SKP_CLIENT,SKP_PRODUCT,SKP_CONTRACT,SKP_APPLICATION,SKP_CREDIT_TYPE,FLAG_KNOWN_APPLICANT,FLAG_KNOWN_CLIENT,FLAG_TOP_UP,NAME_FIRST,NAME_LAST,NAME_MIDDLE,NAME_MOTHER,FLAG_ONLINE_MCL,PRODUCT_TYPE,FLAG_MC_ELIGIBLE,FLAG_ESIGN,POS_GROUP,FLAG_SOLD,FLAG_POSITIVE_OFFER,ORIGINAL_AMT_PAYMENT_CASH,FLAG_EXISTING_SC,FLAG_MOB_APP,DATE_DECISION,FLAG_WORKING_DAY_DECISION,RATE_PAYMENT_CASH_MIN,EXTRA_DP_RATE,EXTRA_DP_AMOUNT,NAME_CLASS_STATUS,ANON_SKP_CREDIT_CASE,AMT_CREDIT_NO_VAS,RISK_GRADE_CBN_POS,AMT_ANNUITY_WO_FEE,CANCELLATION_TYPE,TEXT_CANCELLATION_REASON,RG_PD,RG_SIMULATED,COLUMN1,COLUMN2,FLAG_RBP,FLAG_AFTER_RBP_ROLL_OUT,FLAG_RBP_WAVE_DATE,FLAG_RBP_WAVE_AREA,SNAPP_LOAN_APP_NO,LIN_SCORE_UNIVERSAL,PD_SCORE_UNIVERSAL,UNIVERSAL_RISK_GRADE,FIRST_BOD_CHANNEL,SECOND_BOD_CHANNEL,LIN_SCORE_UNIVERSAL_OLD,PD_SCORE_UNIVERSAL_OLD,UNIVERSAL_RISK_GRADE_OLD,FLAG_GUARANTEED_LIMIT,ID_CUID_UPDATED,SKP_CLIENT_UPDATED,REJECT_REASON,REJECTION_STAGE,REJECT_REASON_OWNER,REJECT_REASON_TOTAL,FLAG_PILOT_BVM,REL_RG,PRODUCT_TYPE_V2,PRERESULT_FANT_TYPE,PRERESULT_FANT_CODE,ATTRIBUTION_TYPE
0,7620828797,103513514271,2591125263,Cancelled,Other,2025-01-24 00:00:00,2025-01-20 00:00:00,2025-01-01 00:00:00,20,1,1,1,1,nan,nan,nan,F,XNA,Married,OWNED,1.0,1975-12-29 00:00:00,49.0,2013-07-01 00:00:00,City of Talisay,Government employee,OTHER,HEALTH,15000.0,IM_10+_17,0.0,NaN,0.0,#Mobile Paperless,01429,STAND,GMA Virtual Salesroom,NaN,THYGSTU5I,Koyal_User,NaN,nan,nan,nan,nan,*,*,*,*,*,*,*,NaN,*,*,*,*,*,XNA,XNA,XNA,XNA,NaN,0,0,0,0,1,0,0,0,Y,Koyal_User,NaN,NaN,NaN,No remittance,NaN,0,0,0,0,0,NaN,OTHER,nan,XNA,nan,nan,5270932435,-1,-79,14758352156,-1,0,0.0,0,CHERYL,*,SEDERIO,*,0,POS,NaN,0,XNA,NaN,NaN,NaN,0,1,2025-01-24 00:00:00,1,-1.0,NaN,NaN,Acquisition,2c7bb64f-d2cf-26fc-e063-2206480ad9e7,0.0,nan,NaN,Cif,CUID_INCONSISTENCY,N/A,nan,NaN,NaN,0.0,NaN,nan,NON-RBP,NaN,NaN,NaN,nan,GMA,nan,NaN,NaN,NaN,NaN,2591125263,5270932435,nan,nan,nan,NaN,NaN,NaN,XNA,nan,nan,nan
1,7620828859,103513236960,3261970684,In Preprocess,Other,2025-01-24 00:00:00,2025-01-20 00:00:00,2025-01-01 00:00:00,13,1,1,1,1,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,NaN,NaN,NaN,#Mobile Paperless,01429,STAND,GMA Virtual Salesroom,NaN,THYGSTU5I,nan,NaN,nan,nan,nan,nan,*,*,*,*,*,*,*,NaN,*,*,*,*,*,XNA,XNA,XNA,XNA,NaN,0,0,1,0,0,0,0,0,N,nan,NaN,NaN,NaN,No remittance,NaN,0,0,0,0,0,NaN,OTHER,nan,XNA,nan,nan,5903864491,-1,-79,-43,-1,0,0.0,0,nan,*,nan,*,0,POS,NaN,0,XNA,NaN,NaN,NaN,0,1,nan,1,NaN,NaN,NaN,nan,2c8ff705-3a0b-940c-e063-2206480a4bab,NaN,nan,NaN,nan,nan,N/A,nan,NaN,NaN,0.0,NaN,nan,NON-RBP,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,3261970684,5903864491,nan,nan,nan,NaN,NaN,NaN,POS,nan,nan,nan
2,7620829014,103513485981,3791563137,In P

## Process Data

In [46]:
insert_row_to_sql(df,df_base,schema,table_name,source_owner,source_table)

Fetching Oracle data types.
Time elapsed: 0.397961
Fetching python data types.
Time elapsed: 5.886757
Converting to sqlalchemy data types.
Commit.
Time elapsed: 1745.589588


# Bulk Insert Into SQL Table

In [47]:
bulk_insert_row_to_sql(df,df_base,schema,table_name)

Fetching Oracle data types.
Time elapsed: 0.616741
Fetching python data types.
Time elapsed: 8.859248
Converting to sqlalchemy data types.
Commit.
Execution time: 57.548907 s


# Merge Into SQL Table

In [48]:
connection_key = 'text_contract_number'
merge_to_sql(df_base,table_name,schema,query,connection_key)

Merged successfully. Commit.
Time elapsed: 5.931787


# Variable Merge Into SQL Table

In [49]:
query_c_date = f'''
with tab_base as
(
select /*+ materialize */
       dm.id_cuid
      -- Basic Scoring
      ,case
            when dm.text_contract_number is not null
            then 1 else 0
       end                                                                      flag_with_app
      ,dm.text_contract_number
      ,dm.date_application
      ,dm.pure_population
      ,dm.pure_population_pre
      ,dm.prelim1_strategy_type
      ,dm.risk_grade_prelim1
      ,dm.flag_with_offer_v2_prelim1
      ,dm.flag_with_offer_v2
      -- DPOS Tags
      ,dm.flag_with_dpos_offer_prelim1
      ,dm.flag_with_dpos_offer
      ,dm.code_prm_decision
      ,case when bs.text_contract_number is not null then 1
            when dm.text_contract_number is not null then 0
       end                                                                      flag_dpos_app
      -- Basic Scoring Decision
      ,dm.cnt_approve                                                           flag_approve
      ,dm.cnt_decided                                                           flag_decided
      ,dm.cnt_signed                                                            flag_signed
      -- Basic Scoring Scores
      ,sa.*
from ap_risk.pos_ec_datamart_tree dm
left join ap_risk.dpos_basic_scoring bs
       on bs.text_contract_number = dm.text_contract_number
      and bs.date_application >= c_date_start
      and bs.date_application <  c_date_end
left join ap_risk.uw_bi_scorecard_dtm sa
       on sa.skp_credit_case = dm.skp_credit_case
      and sa.date_decision >= c_date_start
      and sa.date_decision <  c_date_end
      and sa.scorecard_name in (
                               'EC_202201_GMv8_Main_Scorecard'
                              ,'EC_202110_GMv8_Prelim_Scorecard'
                              ,'EC_202301_GMv10_Prelim_Scorecard'
                              ,'EC_202301_GMv10_Main_Scorecard'
                              )
where 1=1
      and dm.date_decision >= c_date_start
      and dm.date_decision <  c_date_end
)

, tab_gm_pillar as
(
select
       tb.*
from tab_base
unpivot(
        (pillar_name,num_score_value_linear_pillar,num_score_value_pillar)
        for num_pillar_code
        in (
            (pillar_1,   pillar_value_linear_1,  pillar_value_1)    as '1'
           ,(pillar_2,   pillar_value_linear_2,  pillar_value_2)    as '2'
           ,(pillar_3,   pillar_value_linear_3,  pillar_value_3)    as '3'
           ,(pillar_4,   pillar_value_linear_4,  pillar_value_4)    as '4'
           ,(pillar_5,   pillar_value_linear_5,  pillar_value_5)    as '5'
           ,(pillar_6,   pillar_value_linear_6,  pillar_value_6)    as '6'
           ,(pillar_7,   pillar_value_linear_7,  pillar_value_7)    as '7'
           ,(pillar_8,   pillar_value_linear_8,  pillar_value_8)    as '8'
           ,(pillar_9,   pillar_value_linear_9,  pillar_value_9)    as '9'
           )
       ) tb
)

select /*+ PARALLEL(scom,4) PARALLEL(scop,4) */
       count(1) over(partition by scom.id_cuid, scom.skp_credit_case) cnt_predictor_in_skp
      ,count(1) over(partition by scom.id_cuid, scom.skp_credit_case, scom.scorecard_name) cnt_predictor_in_sco
      ,count(1) over(partition by scom.id_cuid, scom.skp_credit_case, scom.scorecard_name, scom.pillar_name) cnt_predictor_in_pil
      ,scom.*
      ,scop.num_predictor_code
      ,scop.predictor_name
      ,scop.predictor_value                                                     num_score_value_linear_predictor
      ,trunc(1/(1+exp(-(scop.predictor_value))),6)                              num_score_value_predictor
      ,scop.predictor_bin
from tab_gm_pillar scom
left join (
            select * from ap_risk.uw_bi_scorecard_predictors_dtm
            unpivot(
                    (predictor_name,predictor_value,predictor_bin)
                    for num_predictor_code
                    in (
                        (predictor_1,   predictor_score_1,  predictor_bin_1)    as '1'
                       ,(predictor_2,   predictor_score_2,  predictor_bin_2)    as '2'
                       ,(predictor_3,   predictor_score_3,  predictor_bin_3)    as '3'
                       ,(predictor_4,   predictor_score_4,  predictor_bin_4)    as '4'
                       ,(predictor_5,   predictor_score_5,  predictor_bin_5)    as '5'
                       ,(predictor_6,   predictor_score_6,  predictor_bin_6)    as '6'
                       ,(predictor_7,   predictor_score_7,  predictor_bin_7)    as '7'
                       ,(predictor_8,   predictor_score_8,  predictor_bin_8)    as '8'
                       ,(predictor_9,   predictor_score_9,  predictor_bin_9)    as '9'
                       ,(predictor_10,  predictor_score_10, predictor_bin_10)   as '10'
                       ,(predictor_11,  predictor_score_11, predictor_bin_11)   as '11'
                       ,(predictor_12,  predictor_score_12, predictor_bin_12)   as '12'
                       ,(predictor_13,  predictor_score_13, predictor_bin_13)   as '13'
                       ,(predictor_14,  predictor_score_14, predictor_bin_14)   as '14'
                       ,(predictor_15,  predictor_score_15, predictor_bin_15)   as '15'
                       ,(predictor_16,  predictor_score_16, predictor_bin_16)   as '16'
                       ,(predictor_17,  predictor_score_17, predictor_bin_17)   as '17'
                       ,(predictor_18,  predictor_score_18, predictor_bin_18)   as '18'
                       )
                   )
            where 1=1
                  and date_decision >= c_date_start
                  and date_decision <  c_date_end
          ) scop
       on scop.skp_credit_case = scom.skp_credit_case
      and scop.date_decision = scom.date_decision
      and scop.pillar_name = scom.pillar_name
'''
c_date_start = "DATE '2025-01-24'"
c_date_end = "DATE '2025-01-30'"

source_owner = 'AP_RISK'
source_table = 'AH_RBB_POS_EC_SCORECARD_MONITORING'
connection_key = ['skp_credit_case','scorecard_name','pillar_name','predictor_name']

timeout = 60 # default = 60 seconds
temporal = 'days' # days, weeks, months, years
increment = 1

## Process Data

In [50]:
var_merge_to_sql(schema,source_table,source_owner,
                 query_c_date,c_date_start,c_date_end,
                 connection_key,timeout,temporal,increment)

Fetching Oracle data types.
Time elapsed: 0.496798
Fetching python data types.
Time elapsed: 0.128201
Converting to sqlalchemy data types.
Dataframe created. Time elapsed: 6.341520 s
Execution time: 6.341520 s
Timeline: DATE '2025-01-24' to DATE '2025-01-28'
Process takes too long. Partitioning timeline by day and populating individually.
Time elapsed: 60.170243
Timeline: DATE '2025-01-24' to DATE '2025-01-25'
Merged successfully. Commit.
Time elapsed: 1580.347120
Timeline: DATE '2025-01-25' to DATE '2025-01-26'
Merged successfully. Commit.
Time elapsed: 613.534463
Timeline: DATE '2025-01-26' to DATE '2025-01-27'
Merged successfully. Commit.
Time elapsed: 560.414301
Timeline: DATE '2025-01-27' to DATE '2025-01-28'
Merged successfully. Commit.
Time elapsed: 506.070499
Chunks merge done. Time elapsed: 3260.369403 s
